In [1]:
# Optimal parameters

k = 4
r = 1
alpha = None
eta = None

In [2]:
# Read input text

from gensim.corpora.dictionary import Dictionary
from lda_helpers import read_lda_input  # Package with helpers

title_texts = read_lda_input('lda_input/lda_input.jl', title=True)
texts = [title_text[1] for title_text in title_texts]
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

In [8]:
# Run model

from lda_helpers import get_lda_model

lda_model = get_lda_model(corpus, id2word, k, r, alpha, eta)

NameError: name 'lda_best' is not defined

In [ ]:
# Save trained model

lda_model.save('lda_output/lda_model')

In [9]:
# Display topics

import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
LDAvis_display = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_display

C:\Users\Mihir\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.192682  0.040824       1        1  38.069493
3     -0.069241 -0.196001       2        1  35.433015
1      0.040031  0.019311       3        1  13.724114
0     -0.163472  0.135866       4        1  12.773377, topic_info=        Term         Freq        Total Category  logprob  loglift
383   school  1596.000000  1596.000000  Default  30.0000  30.0000
361     high   963.000000   963.000000  Default  29.0000  29.0000
548  student   935.000000   935.000000  Default  28.0000  28.0000
325    power   626.000000   626.000000  Default  27.0000  27.0000
154    world  1473.000000  1473.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
991     girl   114.310803  1212.708841   Topic4  -5.4771  -0.3039
281    story    88.210105   404.755105   Topic4  -5.7363   0.5342
496    raise    63.086259    98.561326   Topic4  -6.0715   1.6116
357   follow    82.309609   553.499722   Topic4  -5.8055   0.1520
74      time    65.218254   679.724160   Topic4  -6.0383  -0.2861

[299 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
656       1  0.861382  ability
656       2  0.127612  ability
656       3  0.006381  ability
656       4  0.003190  ability
519       1  0.124311  academy
...     ...       ...      ...
80        3  0.123790    young
80        4  0.054069    young
1780      2  0.988197  younger
2988      2  0.014669     yuki
2988      4  0.982827     yuki

[560 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1])

In [5]:
# Write Genre Names to file

import json

k_names = ['Potato', 'Lettuce', 'Cabbage', 'Broccoli']

with open('lda_output/genre_names.jl', 'w') as f:
    for i in range(k):
        # Write output JSON as newline
        record = {
            'LDA Genre ID': i,
            'LDA Genre Name': k_names[i]
        }
        line = json.dumps(record)
        f.write('{}\n'.format(line))

In [6]:
# Write each Genre's word-distribution to file

with open('lda_output/genre_word_weights.jl', 'w') as f:
    for i in range(k):
        genre = lda_model.show_topic(i, topn=30)
        for word, word_weight in genre:
            # Write output JSON as newline
            record = {
                'LDA Genre ID': i,
                'Word': word,
                'Weight': float(word_weight)
            }
            line = json.dumps(record)
            f.write('{}\n'.format(line))

In [7]:
# Write each Anime's genre-breakdown to file

with open('lda_output/anime_genre_weights.jl', 'w') as f:
    for i, bow in enumerate(corpus):
        title = title_texts[i][0]
        anime_genres = lda_model.get_document_topics(bow, minimum_probability=0)
        for genre_id, genre_weight in anime_genres:
            # Write output JSON as newline
            record = {
                'Title': title,
                'LDA Genre ID': genre_id,
                'LDA Genre Weight': float(genre_weight)
            }
            line = json.dumps(record)
            f.write('{}\n'.format(line))